In [1]:
# Importar bibliotecas necessárias
import json
import os
import math
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Definir caminhos e parâmetros
DATASET_PATH = "../data/genres_original/"  # Caminho para o dataset de gêneros originais
JSON_PATH = "../data/data_10.json"        # Caminho para salvar o arquivo JSON
SAMPLE_RATE = 22050                        # Taxa de amostragem
TRACK_DURATION = 30                         # Duração de cada faixa em segundos
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION  # Número de amostras por faixa

# Número de segmentos que cada faixa de áudio será dividida
NUM_SEGMENTS = 10

# Parâmetros para extração das MFCCs
NUM_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 512


In [3]:
def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=10):
    """Extrai as MFCCs do dataset de música e as salva em um arquivo JSON junto com os rótulos de gêneros.
    
    :param dataset_path (str): Caminho para o dataset de gêneros musicais.
    :param json_path (str): Caminho para salvar o arquivo JSON.
    :param num_mfcc (int): Número de coeficientes MFCC a serem extraídos.
    :param n_fft (int): Número de amostras por FFT.
    :param hop_length (int): Número de amostras entre sucessivas transformadas FFT.
    :param num_segments (int): Número de segmentos em que cada faixa de áudio será dividida.
    """
    
    # Dicionário para armazenar os dados
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }
    
    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)
    
    # Lista para registrar arquivos problemáticos
    problematic_files = []
    
    # Loop através de todas as pastas de gênero
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # Ignorar a pasta principal
        if dirpath != dataset_path:
            # Extrair o rótulo do gênero a partir do nome da pasta
            semantic_label = os.path.basename(dirpath)
            data["mapping"].append(semantic_label)
            print(f"\nProcessing Genre: {semantic_label}")
            
            # Processar todos os arquivos de áudio na pasta de gênero
            for f in filenames:
                file_path = os.path.join(dirpath, f)
                try:
                    # Carregar o arquivo de áudio
                    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
                    
                    # Verificar se a duração do sinal corresponde ao esperado
                    if len(signal) < SAMPLES_PER_TRACK:
                        print(f"File {file_path} is shorter than expected. Skipping.")
                        problematic_files.append(file_path)
                        continue
                    
                    # Processar todos os segmentos de áudio
                    for d in range(num_segments):
                        # Calcular o início e o fim das amostras para o segmento atual
                        start_sample = samples_per_segment * d
                        end_sample = start_sample + samples_per_segment
                        
                        # Extrair as MFCCs - Argumento nomeado 'y' adicionado
                        mfcc = librosa.feature.mfcc(y=signal[start_sample:end_sample],
                                                    sr=sr,
                                                    n_mfcc=num_mfcc,
                                                    n_fft=n_fft,
                                                    hop_length=hop_length)
                        mfcc = mfcc.T  # Transpor para que cada linha represente um vetor de MFCC
                        
                        # Verificar se o número de vetores de MFCC está correto
                        if len(mfcc) == num_mfcc_vectors_per_segment:
                            data["mfcc"].append(mfcc.tolist())
                            data["labels"].append(i-1)  # Ajustar o índice do rótulo
                        else:
                            print(f"Segment {d+1} in file {file_path} has an unexpected number of MFCC vectors. Skipping.")
                            problematic_files.append(file_path)
                            
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
                    problematic_files.append(file_path)
    
    # Salvar os dados extraídos no arquivo JSON
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)
    
    print("\nMFCCs have been successfully saved to JSON file!")
    
    # Salvar os arquivos problemáticos em um log
    if problematic_files:
        with open("../data/problematic_files.log", "w") as log_file:
            for file in problematic_files:
                log_file.write(f"{file}\n")
        print(f"\nFound {len(problematic_files)} problematic files. Details are saved in 'problematic_files.log'.")
    else:
        print("\nNo problematic files found.")


In [4]:
# Executar a função para extrair e salvar as MFCCs
save_mfcc(DATASET_PATH, JSON_PATH, num_mfcc=NUM_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH, num_segments=NUM_SEGMENTS)



Processing Genre: blues

Processing Genre: classical
File ../data/genres_original/classical\classical.00049.wav is shorter than expected. Skipping.
File ../data/genres_original/classical\classical.00051.wav is shorter than expected. Skipping.

Processing Genre: country
File ../data/genres_original/country\country.00003.wav is shorter than expected. Skipping.
File ../data/genres_original/country\country.00004.wav is shorter than expected. Skipping.
File ../data/genres_original/country\country.00007.wav is shorter than expected. Skipping.

Processing Genre: disco
File ../data/genres_original/disco\disco.00014.wav is shorter than expected. Skipping.

Processing Genre: hiphop
File ../data/genres_original/hiphop\hiphop.00031.wav is shorter than expected. Skipping.
File ../data/genres_original/hiphop\hiphop.00032.wav is shorter than expected. Skipping.

Processing Genre: jazz


C:\Users\ianfe\AppData\Local\Temp\ipykernel_25720\1903647368.py:39: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
c:\Users\ianfe\Desktop\music bot\env\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error processing ../data/genres_original/jazz\jazz.00054.wav: 

Processing Genre: metal

Processing Genre: pop

Processing Genre: reggae

Processing Genre: rock
File ../data/genres_original/rock\rock.00038.wav is shorter than expected. Skipping.

MFCCs have been successfully saved to JSON file!

Found 10 problematic files. Details are saved in 'problematic_files.log'.


In [5]:
# Verificar o conteúdo do arquivo JSON
import json

with open(JSON_PATH, "r") as fp:
    data = json.load(fp)

print(f"\nNúmero de gêneros mapeados: {len(data['mapping'])}")
print(f"Número total de segmentos de áudio: {len(data['mfcc'])}")
print(f"Número total de rótulos: {len(data['labels'])}")



Número de gêneros mapeados: 10
Número total de segmentos de áudio: 9900
Número total de rótulos: 9900
